In [29]:
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, Image, Paragraph, Spacer, TableStyle
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import cm
import pandas as pd
import helper_functions as hf
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# Register the emoji font (replace 'Symbola.ttf' with the path to your font file)
pdfmetrics.registerFont(TTFont('Emoji', hf.DATAPATH.joinpath("assets/NotoEmoji-Regular.ttf")))

# Create a new style that uses the emoji font
emoji_style = getSampleStyleSheet()['Normal']
emoji_style.fontName = 'Emoji'


In [31]:

fpath = hf.DATAPATH.joinpath("hidden/nickname_to_name.csv")
clear_names = pd.read_csv(fpath)
clear_name_dict = clear_names.set_index("nickname")["name"].to_dict()
df = hf.get_players().sort_values(["institute", "nickname"]).fillna("")

# df[df["nickname"] == "Pushy Bulldog"]


In [32]:

def hex_to_rgb(hex):
    hex = hex.strip("#")
    return tuple(int(hex[i:i+2], 16)for i in (0, 2, 4))

team_colors = {f"Team {let}": colors.Color(*hex_to_rgb(hf.Team.colors[i]), alpha=0.15) for i, let in enumerate("ABC")}

organizers = {o.nickname: [hf.SPORTS_EVENTS[key].icon for key in o.sport_keys] for o in hf.SPORTS_ORGANIZERS.values()}


In [33]:

# Create a PDF with A4 page size
marg = 0.2*cm
doc = SimpleDocTemplate(str(hf.DATAPATH.joinpath("hidden/nametags.pdf")), pagesize=A4, leftMargin=marg, rightMargin=marg, topMargin=marg, bottomMargin=marg)
# Create a list to hold the table data
data = []
avatar_size = 2*cm
inst_size = 0.8*cm
style = TableStyle()
# Loop over your list of names and avatars
for i in range(0, len(clear_names), 2):
    row = []
    for j in range(2):
        if i + j < len(clear_names):
            entry = df.iloc[i+j]
            nickname = entry["nickname"]
            team = entry["Team"]
            institute = entry["institute"]
            sports_attending = [sport.icon for sport_key, sport in hf.SPORTS_EVENTS.items() if entry[f"subteam_{sport_key}"] != ""]
            if len(sports_attending) == 0:
                sports_attending = [sport.icon for sport_key, sport in hf.SPORTS_EVENTS.items() if entry[sport_key]]
            true_name = clear_name_dict[nickname]
            impath = hf.FpathRegistry.get_animal_pic_path(nickname, False)
            # Create an Image object for the avatar
            avatar = Image(impath, width=avatar_size, height=avatar_size)
            impath = hf.FpathRegistry.get_institute_pic_path(institute)
            institute_pic = Image(impath, width=inst_size, height=inst_size)
            
            # Create a Paragraph object for the name
            # name = Paragraph(nickname, emoji_style)
            # Create a Paragraph object for the true name
            true_name_text = f"<strong>{true_name}</strong>" + f" ({team})"
            nickname_text = f"<i>{nickname}</i>"
            # Create a Paragraph object for the nickname
            if nickname in organizers:
                sports_hosting = "".join(organizers[nickname])
                nickname_text += f" <span name='Emoji'>ℹ️</span> [<span name='Emoji'>{sports_hosting}</span>]"
                true_name_text = f"<span name='Emoji'>⭐</span> {true_name_text}"
                if nickname == "Exemplary Cassowary":
                    true_name_text = true_name_text.replace("Team ", "")
            true_name_paragraph = Paragraph(true_name_text, getSampleStyleSheet()['Normal'])

            hf.SPORTS_ORGANIZERS
            # Create a line
            line = Spacer(1, 0.1*cm)
            line.hAlign = 'CENTER'
            line.getSpaceBefore = lambda: 0.1*cm
            line.getSpaceAfter = lambda: 0.1*cm
            line.drawOn = lambda canvas, x, y: canvas.line(x, y, x+6 *cm, y)
            nickname_paragraph = Paragraph(nickname_text, getSampleStyleSheet()['Normal'])

            sports_paragraph = Paragraph("-".join(sports_attending), emoji_style)
            
            name_style = TableStyle()
            name_style.add('TOPPADDING', (0, 0), (-1, -1), 1)
            name_style.add('BOTTOMADDING', (0, 0), (-1, -1), 1)
            # Create a table for the true name, line, and nickname
            name_table = Table([[true_name_paragraph], [line], [nickname_paragraph], [sports_paragraph]])
            name_table.setStyle(name_style)

            # Add a 'BACKGROUND' command for this cell to the style
            style.add('BACKGROUND', (j, i//2), (j, i//2), team_colors.get(team, colors.white))
            style.add('TOPPADDING', (j, i//2), (j, i//2), 1)
            style.add('BOTTOMADDING', (j, i//2), (j, i//2), 1)
            # Add the avatar and name to the row
            inner_table = Table([[avatar, name_table, institute_pic]],
                             colWidths=[avatar_size, 6.7*cm, inst_size])
            inner_table.vAlign = "MIDDLE"
            row.append(inner_table)
        else:
            # If there are no more names, add an empty string
            row.append('')
    # Add the row to the table data
    data.append(row)

# Create a Table object
table = Table(data)

# Set the style on the table
table.setStyle(style)
# Add the table to the elements to be added to the PDF
elements = []
elements.append(table)

# Build the PDF
doc.build(elements)
